In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests
from typing import Optional

load_dotenv()


def get_attendance(from_to: Optional[tuple[str, str]] = None) -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "uuid": attendee["uuid"],
                "displayName": attendee["displayName"],
                **{
                    entry["date"]["isoYearMonthDay"]: entry["isMarkedAttended"]
                    for entry in attendee["entries"] or []
                    if pd.to_datetime(entry["date"]["isoYearMonthDay"]).day
                    in set(range(8, 15)).union(range(22, 29))
                },
            }
            for attendee in requests.get(
                os.getenv("ATTENDANCE_URL", "")
                + (f"/start/{from_to[0]}/end/{from_to[1]}" if from_to else ""),
                {"lang": "eng"},
                headers={
                    "Cookie": os.getenv("ATTENDANCE_COOKIE", ""),
                },
            ).json()["attendanceData"]["attendees"]
            if from_to
            or "becef79e-6486-4c26-a9c8-1a308448c33d" in attendee["unitOrgsCombined"]
        ]
    ).drop(columns=["displayName"] if from_to else [])


attendance = get_attendance().merge(
    get_attendance(("2024-04-01", "2024-11-01")), on="uuid", suffixes=("", "_duplicate")
)
attendance.drop(
    columns=[col for col in attendance.columns if col.endswith("_duplicate")],
    inplace=True,
)
attendance.drop(columns=["uuid"], inplace=True)
attendance.set_index("displayName", inplace=True)
attendance.sort_index(inplace=True)
attendance.sort_index(axis=1, inplace=True)
attendance

In [ ]:
attendance.T[(attendance.T.mean() - attendance.T.ewm(span=5).mean()[-1:].T.squeeze()).sort_values(ascending=False).head(10).index]